# Anthropic Example

First let's install the required packages

In [ ]:
%pip install -U anthropic
%pip install -U agentops

Then import them

In [ ]:
import anthropic
import agentops
import os
from dotenv import load_dotenv

Next, we'll grab our API keys. You can use dotenv like below or however else you like to load environment variables

In [ ]:
load_dotenv()
ANTHROPIC_API_KEY = os.getenv("ANTHROPIC_API_KEY") or "<your_anthropic_key>"
AGENTOPS_API_KEY = os.getenv("AGENTOPS_API_KEY") or "<your_agentops_key>"

In [ ]:
agentops.init(AGENTOPS_API_KEY, default_tags=["anthropic-example"])
client = anthropic.Anthropic()

In [ ]:
stream = client.messages.create(
    max_tokens=1024,
    model="claude-3-opus-20240229",
    messages=[
        {
            "role": "user",
            "content": "What is 2+2?",
        }
    ],
    stream=True,
)

response = ""
for event in stream:
    if event.type == "content_block_delta":
        response += event.delta.text
    elif event.type == "message_stop":
        print("\n")
        print(response)
        print("\n")

In [ ]:
stream = client.messages.create(
    max_tokens=1024,
    model="claude-3-opus-20240229",
    tools=[
        {
            "name": "web_search",
            "description": "Search the web for information",
            "input_schema": {
                "type": "object",
                "properties": {
                    "query": {
                        "type": "string",
                        "description": "The search query"
                    }
                },
                "required": ["query"]
            }
        }
    ],
    messages=[
        {
            "role": "user",
            "content": "Tell me everything you can about AgentOps",
        }
    ],
    stream=True,
)

response = ""
for event in stream:
    if event.type == "content_block_delta":
        if event.delta.type == "text":
            response += event.delta.text
    elif event.type == "message_stop":
        print("\n")
        print(response)
        print("\n")

In [ ]:
agentops.end_session("Success")